In [1]:
import os
import json
import pandas as pd
from cog_analysis import load_boat_data, analyze_session
from report_fct import filter_interval, compute_directional_gain


def build_csv_from_summary(summary_path, data_root, output_csv="all_data.csv"):
    with open(summary_path, "r") as f:
        summary = json.load(f)

    all_rows = []

    for run_entry in summary:
        run_name = run_entry["run"]
        intervals = run_entry["intervals"]

        # Recherche du dossier de la run
        run_path = None
        for root, dirs, files in os.walk(data_root):
            if os.path.basename(root) == run_name:
                run_path = root
                break

        if not run_path:
            print(f"⚠️ Run folder not found for: {run_name}")
            continue

        # Chargement des fichiers CSV
        csv_files = [f for f in os.listdir(run_path) if f.endswith(".csv")]
        if len(csv_files) != 2:
            print(f"⚠️ Skipping {run_name}: expected 2 CSVs, found {len(csv_files)}")
            continue

        csv_paths = [os.path.join(run_path, f) for f in csv_files]

        try:
            df1, df2, name1, name2 = load_boat_data(csv_paths[0], csv_paths[1])
            if df1.empty or df2.empty:
                continue
        except Exception as e:
            print(f"❌ Error loading CSVs for {run_name}: {e}")
            continue

        # Traitement des intervalles
        for i, interval in enumerate(intervals):
            start, end = interval["start_time"], interval["end_time"]
            if end - start < 30:
                print(f"⚠️ Skipping interval {i + 1} for {run_name}: duration < 30 seconds")
                continue

            df1_clip = filter_interval(df1, start, end)
            df2_clip = filter_interval(df2, start, end)
            if df1_clip.empty or df2_clip.empty:
                print(f"⚠️ Skipping interval {i + 1} for {run_name}: no data in interval")
                continue

            # Définir les rôles master/slave
            master_df, slave_df = (df1_clip, df2_clip)
            if not interval.get("boat1_master_leeward", False):
                master_df, slave_df = df2_clip, df1_clip

            # Construction des données ligne par ligne
            for df_clip, prefix, other_prefix in [(df1_clip, "boat1", "boat2"), (df2_clip, "boat2", "boat1")]:
                df = df_clip.copy()
                df["run"] = run_name
                df["interval_id"] = i + 1
                df["boat_name"] = interval.get(f"{prefix}_name", "")
                df["opponent_name"] = interval.get(f"{other_prefix}_name", "")
                df["boat_role"] = "master" if interval.get(f"{prefix}_master_leeward", False) else "slave"
                df["boat_weight"] = interval.get(f"{prefix}_total_weight", None)
                df["interval_duration"] = interval.get("duration", None)
                df["mast_brand"] = interval.get(f"{prefix}_mast_brand", None)

                # Calculs des gains
                gain_forward = []
                gain_lateral = []
                gain_vmg = []

                for t in df["SecondsSince1970"]:
                    m_clip = master_df[master_df["SecondsSince1970"] <= t]
                    s_clip = slave_df[slave_df["SecondsSince1970"] <= t]
                    gain_df = compute_directional_gain(m_clip, s_clip)

                    if gain_df.empty:
                        gain_forward.append(np.nan)
                        gain_lateral.append(np.nan)
                        gain_vmg.append(np.nan)
                    else:
                        gain_forward.append(gain_df.loc["Total Gain", "Forward"])
                        gain_lateral.append(gain_df.loc["Total Gain", "Lateral"])
                        gain_vmg.append(gain_df.loc["Total Gain", "VMG"])

                df["gain_forward"] = gain_forward
                df["gain_lateral"] = gain_lateral
                df["gain_vmg"] = gain_vmg
                df["side_line2"] = df["Line_L"] + df["Line_R"]
                df["total_line2"] = df["side_line2"] + df["Line_C"]

                all_rows.append(df)

    # Sauvegarde finale
    if not all_rows:
        print("❌ No valid data found.")
        return

    df_global = pd.concat(all_rows, ignore_index=True)
    df_global.to_csv(output_csv, index=False)
    print(f"✅ Global CSV saved to: {output_csv}")


In [2]:
build_csv_from_summary(
    summary_path="summary_enriched.json",
    data_root="../Data_Sailnjord/Straight_lines",
    output_csv="all_data.csv"
)


✅ Global CSV saved to: all_data.csv
